In [ ]:
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Get the file path from an environment variable
RAW_DATA_PATH = os.getenv("RAW_DATA_PATH")

# Load data
data = pd.read_csv(RAW_DATA_PATH)

# Convert timestamp column to datetime type
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Extract week and month for filtering
data['Week'] = data['Timestamp'].dt.isocalendar().week
data['Month'] = data['Timestamp'].dt.month




In [ ]:

# Define weeks of interest
weeks = [41, 42, 43, 44]

# Create a subplot layout with 2 rows, 2 columns
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=[f"Week {w}" for w in weeks], 
                    specs=[[{"type": "domain"}, {"type": "domain"}], 
                           [{"type": "domain"}, {"type": "domain"}]])

# Loop through each week and add a pie chart
for i, week in enumerate(weeks):
    # Filter data for the current week
    week_data = data[data["Week"] == week]
    
    # Aggregate plant health status counts
    week_counts = week_data["Plant_Health_Status"].value_counts().reset_index()
    week_counts.columns = ["Plant_Health_Status", "Count"]

    # Add pie chart to the corresponding subplot
    fig.add_trace(go.Pie(labels=week_counts["Plant_Health_Status"], 
                         values=week_counts["Count"], 
                         name=f"Week {week}"), 
                  row=(i // 2) + 1, col=(i % 2) + 1)

# Update layout
fig.update_layout(title_text="Plant Health Status Distribution (Weeks 41-44)", height=700, width=900)

# Show the figure
fig.show()


In [ ]:
# Define plant IDs of interest
plant_ids = [1, 2, 3, 4]

# Create a subplot layout with 2 rows, 2 columns
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=[f"Plant ID {pid}" for pid in plant_ids], 
                    specs=[[{"type": "domain"}, {"type": "domain"}], 
                           [{"type": "domain"}, {"type": "domain"}]])

# Loop through each Plant ID and add a pie chart
for i, pid in enumerate(plant_ids):
    # Filter data for the current plant over the entire month
    plant_data = data[data["Plant_ID"] == pid]
    
    # Aggregate plant health status counts
    plant_counts = plant_data["Plant_Health_Status"].value_counts().reset_index()
    plant_counts.columns = ["Plant_Health_Status", "Count"]

    # Add pie chart to the corresponding subplot
    fig.add_trace(go.Pie(labels=plant_counts["Plant_Health_Status"], 
                         values=plant_counts["Count"], 
                         name=f"Plant ID {pid}"), 
                  row=(i // 2) + 1, col=(i % 2) + 1)

# Update layout
fig.update_layout(title_text="Monthly Plant Health Status (Plant ID 1-4)", height=700, width=900)

# Show the figure
fig.show()


In [ ]:
# Select only numeric columns
numeric_data = data.select_dtypes(include=['number'])

# Compute correlation matrix
corr_matrix = numeric_data.corr()

# Plot heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


In [21]:
import os
import pandas as pd
import plotly.graph_objects as go

# Get the file path from an environment variable
RAW_DATA_PATH = os.getenv("RAW_DATA_PATH")

# Load data
if RAW_DATA_PATH and os.path.exists(RAW_DATA_PATH):
    data = pd.read_csv(RAW_DATA_PATH)
    print("Data loaded successfully.")
else:
    raise FileNotFoundError("CSV file not found. Check RAW_DATA_PATH.")

# Convert timestamp column to datetime type
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Extract week and month for filtering
if "Week" not in data.columns:
    data["Week"] = data["Timestamp"].dt.isocalendar().week
    print("Week column created.")

if "Plant_Health_Status" not in data.columns:
    raise KeyError("Column 'Plant_Health_Status' not found in dataset.")

# Get unique weeks from dataset
unique_weeks = sorted(data["Week"].unique())
print("Unique Weeks:", unique_weeks)

# Create initial figure (default to first available week)
initial_week = unique_weeks[0]
initial_data = data[data["Week"] == initial_week]["Plant_Health_Status"].value_counts().reset_index()
initial_data.columns = ["Plant_Health_Status", "Count"]

# Create figure with initial pie chart
fig = go.Figure()

fig.add_trace(go.Pie(
    labels=initial_data["Plant_Health_Status"], 
    values=initial_data["Count"], 
    name=f"Week {initial_week}"
))

# Create dropdown menu for selecting week
dropdown_buttons = []

for week in unique_weeks:
    # Filter data for the selected week
    week_data = data[data["Week"] == week]["Plant_Health_Status"].value_counts().reset_index()
    week_data.columns = ["Plant_Health_Status", "Count"]

    # Add a dropdown button for each week
    dropdown_buttons.append(
        dict(
            label=f"Week {week}",
            method="update",
            args=[{"labels": [week_data["Plant_Health_Status"]], 
                   "values": [week_data["Count"]]}, 
                  {"title": f"Plant Health Status Distribution - Week {week}"}]
        )
    )

# Add dropdown to the layout
fig.update_layout(
    title_text=f"Plant Health Status Distribution - Week {initial_week}",
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
    )]
)

# Show the interactive figure
fig.show()


Data loaded successfully.
Week column created.
Unique Weeks: [np.uint32(40), np.uint32(41), np.uint32(42), np.uint32(43), np.uint32(44)]


In [32]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get the file path from an environment variable
RAW_DATA_PATH = os.getenv("RAW_DATA_PATH")

# Load data
if RAW_DATA_PATH and os.path.exists(RAW_DATA_PATH):
    data = pd.read_csv(RAW_DATA_PATH)
else:
    raise FileNotFoundError("CSV file not found. Check RAW_DATA_PATH.")

# Convert timestamp column to datetime type
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Extract week
data["Week"] = data["Timestamp"].dt.isocalendar().week

# Get unique weeks
unique_weeks = sorted(data["Week"].unique())

# Select key environmental features (excluding Light Intensity)
env_features = [
    "Soil_Moisture", "Ambient_Temperature", "Soil_Temperature", "Humidity", 
    "Soil_pH", "Nitrogen_Level", "Phosphorus_Level", 
    "Potassium_Level", "Chlorophyll_Content", "Electrochemical_Signal"
]

# --------- INITIAL DATA: First Available Week ---------
initial_week = unique_weeks[0]
filtered_data = data[data["Week"] == initial_week]

# Compute Plant Health Distribution (Tile 1)
plant_health_counts = filtered_data["Plant_Health_Status"].value_counts().reset_index()
plant_health_counts.columns = ["Plant_Health_Status", "Count"]

# Compute Avg Environmental Factors (Tile 2)
avg_values = filtered_data[env_features].mean()

# --------- CREATE SUBPLOTS ---------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f"Plant Health Status - Week {initial_week}", 
        f"Avg Environmental Factors - Week {initial_week}"
    ),
    specs=[[{"type": "pie"}, {"type": "bar"}]]
)

# --------- TILE 1: PIE CHART (Plant Health Status) ---------
fig.add_trace(
    go.Pie(
        labels=plant_health_counts["Plant_Health_Status"], 
        values=plant_health_counts["Count"], 
        name=f"Week {initial_week}"
    ),
    row=1, col=1
)

# --------- TILE 2: BAR CHART (Avg Environmental Factors) ---------
fig.add_trace(
    go.Bar(
        x=env_features, 
        y=avg_values, 
        marker_color="lightblue",
        name=f"Week {initial_week}"
    ),
    row=1, col=2
)

# --------- CREATE DROPDOWN MENU ---------
dropdown_buttons = []

for week in unique_weeks:
    filtered_data = data[data["Week"] == week]

    # Compute Plant Health Distribution
    plant_health_counts = filtered_data["Plant_Health_Status"].value_counts().reset_index()
    plant_health_counts.columns = ["Plant_Health_Status", "Count"]

    # Compute Avg Environmental Factors
    avg_values = filtered_data[env_features].mean()

    dropdown_buttons.append(
        dict(
            label=f"Week {week}",
            method="update",
            args=[
                {"labels": [plant_health_counts["Plant_Health_Status"]],  # Update pie chart labels
                 "values": [plant_health_counts["Count"]],  # Update pie chart values
                 "y": [avg_values.tolist()]},  # Update bar chart y-values
                {"title.text": f"Plant Health Status & Environmental Factors - Week {week}"}
            ]
        )
    )

# --------- UPDATE FIGURE LAYOUT ---------
fig.update_layout(
    title_text=f"Plant Health Status & Environmental Factors - Week {initial_week}",
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.17, y=1.15  # Adjust dropdown position
    )]
)

# Show the interactive visualization
fig.show()


In [56]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive_output, widgets, Layout, HBox, VBox

# Get the file path from an environment variable
RAW_DATA_PATH = os.getenv("RAW_DATA_PATH")
if RAW_DATA_PATH is None or not os.path.exists(RAW_DATA_PATH):
    raise FileNotFoundError("RAW_DATA_PATH environment variable is not set or the file is missing.")

# Load data
data = pd.read_csv(RAW_DATA_PATH)

# Convert timestamp column to datetime type
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Week'] = data['Timestamp'].dt.isocalendar().week

# Extract unique weeks and plants
unique_weeks = sorted(data["Week"].unique())
unique_plants = sorted(data["Plant_ID"].unique())

# Function to get pie chart data based on Week & Plant selection
def get_pie_data(week, plant):
    filtered_data = data[(data["Week"] == week) & (data["Plant_ID"] == plant)]
    if filtered_data.empty:
        return pd.DataFrame({"Plant_Health_Status": ["No Data"], "Count": [1]})
    
    pie_data = filtered_data["Plant_Health_Status"].value_counts().reset_index()
    pie_data.columns = ["Plant_Health_Status", "Count"]
    return pie_data

# Function to get environmental factors data
def get_env_factors_data(week, plant):
    filtered_data = data[(data["Week"] == week) & (data["Plant_ID"] == plant)]
    
    if filtered_data.empty:
        return pd.DataFrame({"Factor": ["No Data"], "Value": [0]})
    
    # List of environmental factors
    env_factors = [
        'Soil_Moisture', 'Ambient_Temperature', 'Soil_Temperature', 'Humidity', 
        'Soil_pH', 'Nitrogen_Level', 'Phosphorus_Level', 'Potassium_Level', 
        'Chlorophyll_Content', 'Electrochemical_Signal'
    ]
    
    # Calculate mean values for each factor
    env_data = pd.DataFrame({
        'Factor': env_factors,
        'Value': [filtered_data[factor].mean() for factor in env_factors]
    })
    
    return env_data

# Function to create and update both charts
def update_charts(week, plant):
    # Create figure with 1 row and 2 columns for the two tiles
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f"Plant Health Status ", f"Environmental Factors "),
        specs=[[{"type": "pie"}, {"type": "bar"}]]
    )
    
    # Tile 1: Pie Chart
    pie_data = get_pie_data(week, plant)
    fig.add_trace(
        go.Pie(
            labels=pie_data["Plant_Health_Status"],
            values=pie_data["Count"],
            hole=.3,
            domain=dict(x=[0, 0.45]),
            textinfo='percent',
            showlegend=True,  # Enable legend for pie chart
            name="Health Status"  # Name the legend for clarity
        ),
        row=1, col=1
    )
    
    # Tile 2: Bar Chart
    env_data = get_env_factors_data(week, plant)
    fig.add_trace(
        go.Bar(
            x=env_data["Factor"],
            y=env_data["Value"],
            text=env_data["Value"].round(2),
            textposition='auto'
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text=f"Plant Analysis Dashboard - Week {week}, Plant {plant}",
        height=600,
        width=1200,
        showlegend=True,  # Show the legend overall
        legend=dict(
            x=1.05,  # Position the legend outside the plot
            y=0.5,  # Center the legend vertically
            traceorder='normal',
            orientation='v',
            title="Health Status"  # Title for the legend
        )
    )
    
    # Update y-axis label for the bar chart
    fig.update_yaxes(title_text="Value", row=1, col=2)
    
    # Update x-axis of bar chart to show labels at an angle
    fig.update_xaxes(
        tickangle=45,
        tickmode='array',
        tickvals=env_data["Factor"],
        row=1, col=2
    )
    
    fig.show()

# Create interactive widgets (only once)
week_dropdown = widgets.Dropdown(
    options=[(f'Week {week}', week) for week in unique_weeks],
    value=unique_weeks[0],
    description='Week:',
    layout=Layout(width='200px')
)

plant_dropdown = widgets.Dropdown(
    options=[(f'Plant {plant}', plant) for plant in unique_plants],
    value=unique_plants[0],
    description='Plant:',
    layout=Layout(width='200px')
)

# Create the widget UI
ui = HBox([week_dropdown, plant_dropdown])

# Create the interactive output
out = interactive_output(
    update_charts,
    {'week': week_dropdown, 'plant': plant_dropdown}
)

# Display UI and output
display(VBox([ui, out]))
